In [ ]:
pip install konlpy

     |████████████████████████████████| 19.4MB 48.0MB/s 
     |████████████████████████████████| 450kB 33.4MB/s 
     |████████████████████████████████| 92kB 10.8MB/s 
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [ ]:
import pandas as pd
import re
from konlpy.tag import Okt
from konlpy.utils import pprint
from konlpy.tag import Kkma
from konlpy.utils import pprint
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize
import numpy as np

In [ ]:
df = pd.read_csv('kor.csv',names=['Eng','Kor'])
df

,Eng,Kor
0,Go.,가
1,Hi.,안녕.
2,Run!,뛰어!
3,Run.,뛰어.
4,Who?,누구?
...,...,...
3718,Science fiction has undoubtedly been the inspi...,공상 과학 소설은 의심의 여지 없이 오늘날 존재하는 많은 기술에 영감을 주었어.
3719,I started a new blog. I'll do my best not to b...,난 블로그를 시작했어. 블로그를 초반에만 반짝 많이 하다가 관두는 사람처럼은 되지 ...
3720,I think it's a shame that some foreign languag...,몇몇 외국어 선생님이 한 번도 원어민과 공부해본 적도 없으면서 대학을 나올 수 있었...
3721,If someone who doesn't know your background sa...,만일 네 사정도 잘 모르는 사람이 원어민 같다고 말한다면 그건 그 사람이 네가 원어...


In [ ]:

lengths = df["Eng"].str.len()
argmax = np.where(lengths == lengths.max())[0]
df.iloc[argmax]

,Eng,Kor
3722,Doubtless there exists in this world precisely...,의심의 여지 없이 세상에는 어떤 남자이든 정확히 딱 알맞는 여자와 결혼하거나 그 반...


In [ ]:

df['Eng'].str.len().max()

537

In [ ]:

lengths = df["Kor"].str.len()
argmax = np.where(lengths == lengths.max())[0]
df.iloc[argmax]

,Eng,Kor
3722,Doubtless there exists in this world precisely...,의심의 여지 없이 세상에는 어떤 남자이든 정확히 딱 알맞는 여자와 결혼하거나 그 반...


In [ ]:
train, test = df.iloc[:2000], df.iloc[2000:]

In [ ]:
train = train.to_numpy()
test = test.to_numpy()

In [ ]:

import string

string.punctuation

def remove_punct(text):
    text = "".join([char for char in text if char not in string.punctuation])
    return text

In [ ]:
df['Eng']= df['Eng'].apply(lambda x:remove_punct(x))
df['Kor']= df['Kor'].apply(lambda x:remove_punct(x))

In [ ]:

def tokenize(text):
    tokens = text.split()
    tokens2 = re.split('\W+', text)
    #tokens3 = re.findall(r'\S+', text)
    return tokens

df['Eng'] = df['Eng'].apply(lambda x: tokenize(x.lower()))

In [ ]:
okt= Okt()
def tokenize_kor(text):
    tokens = okt.morphs(text)
    return tokens
df['Kor']= df['Kor'].apply(lambda x: tokenize_kor(x))

In [ ]:
df

,Eng,Kor
0,[go],[가]
1,[hi],[안녕]
2,[run],[뛰어]
3,[run],[뛰어]
4,[who],[누구]
...,...,...
3718,"[science, fiction, has, undoubtedly, been, the...","[공상, 과학, 소설, 은, 의심, 의, 여지, 없이, 오늘날, 존재, 하는, 많은..."
3719,"[i, started, a, new, blog, ill, do, my, best, ...","[난, 블로그, 를, 시작, 했어, 블로그, 를, 초반, 에만, 반짝, 많이, 하다..."
3720,"[i, think, its, a, shame, that, some, foreign,...","[몇몇, 외국어, 선생님, 이, 한, 번, 도, 원어민, 과, 공부, 해본, 적도,..."
3721,"[if, someone, who, doesnt, know, your, backgro...","[만일, 네, 사정, 도, 잘, 모르는, 사람, 이, 원어민, 같다고, 말, 한다면..."


In [ ]:
# Fit tokenizer 
from tensorflow.keras.preprocessing.text import Tokenizer
def create_tokenizer(lines):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(lines)
    return tokenizer

In [ ]:
def max_length(lines):
	return max(len(line.split()) for line in lines)

In [ ]:
df['Eng']

0                                                    [go]
1                                                    [hi]
2                                                   [run]
3                                                   [run]
4                                                   [who]
                              ...                        
3718    [science, fiction, has, undoubtedly, been, the...
3719    [i, started, a, new, blog, ill, do, my, best, ...
3720    [i, think, its, a, shame, that, some, foreign,...
3721    [if, someone, who, doesnt, know, your, backgro...
3722    [doubtless, there, exists, in, this, world, pr...
Name: Eng, Length: 3723, dtype: object

In [ ]:
from nltk import word_tokenize
# prepare english tokenizer
eng_tokenizer = create_tokenizer(df['Eng'].values)
eng_vocab_size = len(eng_tokenizer.word_index) + 1
eng_length = df['Eng'].str.len().max()
print('English Vocabulary Size: %d' % eng_vocab_size)
print('English Max Length: %d' % (eng_length))
# prepare german tokenizer
kor_tokenizer = create_tokenizer(df['Kor'].values)
kor_vocab_size = len(kor_tokenizer.word_index) + 1
kor_length = df['Kor'].str.len().max()
print('Korean Vocabulary Size: %d' % kor_vocab_size)
print('Korean Max Length: %d' % (kor_length))

English Vocabulary Size: 2561
English Max Length: 101
Korean Vocabulary Size: 4150
Korean Max Length: 124


In [ ]:
from keras.preprocessing.sequence import pad_sequences

# encode and pad sequences
def encode_sequences(tokenizer, length, lines):
	# integer encode sequences
	X = tokenizer.texts_to_sequences(lines)
	# pad sequences with 0 values
	X = pad_sequences(X, maxlen=length, padding='post')
	return X

In [ ]:
import tensorflow as tf

In [ ]:
def encode_output(sequences, vocab_size):
	ylist = list()
	for sequence in sequences:
		encoded = tf.keras.utils.to_categorical(sequence, num_classes=vocab_size)
		ylist.append(encoded)
	y = array(ylist)
	y = y.reshape(sequences.shape[0], sequences.shape[1], vocab_size)
	return y

In [ ]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(df, test_size=0.2, random_state = 12)

In [ ]:
train['Eng']

2524               [tom, has, decided, to, study, abroad]
2565              [have, you, ever, been, to, disneyland]
227                                         [how, lovely]
651                                  [i, caught, a, cold]
3474    [my, favorite, city, in, the, whole, world, is...
                              ...                        
3313    [tom, roasted, the, marshmallow, over, the, fire]
3325    [i, didnt, know, tom, was, a, child, psycholog...
1414                        [children, need, to, be, fed]
1691                       [i, havent, kissed, mary, yet]
1867                      [thatll, teach, you, a, lesson]
Name: Eng, Length: 2978, dtype: object

In [ ]:
test

,Eng,Kor
2844,"[tom, usually, drinks, beer, at, parties]","[톰, 은, 보통, 파티, 에서, 맥주, 를, 마셔]"
2791,"[i, shouldve, cooked, the, pork, longer]","[돼지고기, 를, 더, 구웠어야, 했는데]"
2961,"[the, cost, of, the, book, is, five, dollars]","[이, 책, 의, 가격, 은, 5, 달러, 야]"
1932,"[whos, your, favorite, singer]","[좋아하는, 가수, 는, 누구, 예요]"
263,"[start, over]","[다시, 해]"
...,...,...
2307,"[tom, seemed, a, little, dangerous]","[톰, 은, 조금, 위험해, 보였어]"
1538,"[do, you, understand, or, not]","[이해, 한, 거, 야, 안, 한거야]"
1069,"[when, does, tom, leave]","[톰, 은, 언제, 떠나]"
3432,"[is, it, true, that, youre, a, high, school, t...","[네, 가, 고등학교, 선생님, 이라는, 게, 사실, 이니]"


In [ ]:

# prepare training data
trainX = encode_sequences(kor_tokenizer, kor_length, train['Kor'])
trainY = encode_sequences(eng_tokenizer, eng_length, train['Eng'])
trainY = encode_output(trainY, eng_vocab_size) 
# prepare validation data
testX = encode_sequences(kor_tokenizer, kor_length, test['Kor'])
testY = encode_sequences(eng_tokenizer, eng_length, test['Eng'])
testY = encode_output(testY, eng_vocab_size)

In [ ]:
from pickle import load
from numpy import array
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from keras.utils.vis_utils import plot_model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import RepeatVector
from tensorflow.keras.layers import TimeDistributed
from tensorflow.keras.callbacks import ModelCheckpoint


In [ ]:

def define_model(src_vocab, tar_vocab, src_timesteps, tar_timesteps, n_units):
	model = Sequential()
	model.add(Embedding(src_vocab, n_units, input_length=src_timesteps, mask_zero=True))
	model.add(LSTM(n_units))
	model.add(RepeatVector(tar_timesteps))
	model.add(LSTM(n_units, return_sequences=True))
	model.add(TimeDistributed(Dense(tar_vocab, activation='softmax')))
	return model

In [ ]:
model = define_model(kor_vocab_size, eng_vocab_size, 20, 101, 64)

model.compile(optimizer='adam', loss='categorical_crossentropy')
print(model.summary())

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 20, 64)            265600    
_________________________________________________________________
lstm_12 (LSTM)               (None, 64)                33024     
_________________________________________________________________
repeat_vector_6 (RepeatVecto (None, 101, 64)           0         
_________________________________________________________________
lstm_13 (LSTM)               (None, 101, 64)           33024     
_________________________________________________________________
time_distributed_6 (TimeDist (None, 101, 2561)         166465    
Total params: 498,113
Trainable params: 498,113
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
filename = 'model.h5'
checkpoint = ModelCheckpoint(filename, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
model.fit(trainX, trainY, epochs=10, batch_size=64, validation_data=(testX, testY), callbacks=[checkpoint], verbose=2)

Epoch 1/10
47/47 - 45s - loss: 5.3733 - val_loss: 1.3582

Epoch 00001: val_loss improved from inf to 1.35815, saving model to model.h5
Epoch 2/10
47/47 - 35s - loss: 0.6924 - val_loss: 0.5347

Epoch 00002: val_loss improved from 1.35815 to 0.53471, saving model to model.h5
Epoch 3/10
47/47 - 36s - loss: 0.4923 - val_loss: 0.4509

Epoch 00003: val_loss improved from 0.53471 to 0.45086, saving model to model.h5
Epoch 4/10
47/47 - 35s - loss: 0.4346 - val_loss: 0.4164

Epoch 00004: val_loss improved from 0.45086 to 0.41639, saving model to model.h5
Epoch 5/10
47/47 - 35s - loss: 0.4108 - val_loss: 0.4008

Epoch 00005: val_loss improved from 0.41639 to 0.40077, saving model to model.h5
Epoch 6/10
47/47 - 35s - loss: 0.3985 - val_loss: 0.3913

Epoch 00006: val_loss improved from 0.40077 to 0.39126, saving model to model.h5
Epoch 7/10
47/47 - 35s - loss: 0.3910 - val_loss: 0.3858

Epoch 00007: val_loss improved from 0.39126 to 0.38582, saving model to model.h5
Epoch 8/10
47/47 - 35s - loss: 

In [ ]:
# map an integer to a word
def word_for_id(integer, tokenizer):
	for word, index in tokenizer.word_index.items():
		if index == integer:
			return word
	return None
  
# generate target given source sequence
def predict_sequence(model, tokenizer, source):
	prediction = model.predict(source, verbose=0)[0]
	integers = [np.argmax(vector) for vector in prediction]
	target = list()
	for i in integers:
		word = word_for_id(i, tokenizer)
		if word is None:
			break
		target.append(word)
	return ' '.join(target)
 
# evaluate the skill of the model
def evaluate_model(model, tokenizer, sources, raw_dataset):
	actual, predicted = list(), list()
	for i, source in enumerate(sources):
		# translate encoded source text
		source = source.reshape((1, source.shape[0]))
		translation = predict_sequence(model, eng_tokenizer, source)
		raw_target, raw_src = raw_dataset[i]b
    
		if i < 10:
			print('src=[%s], target=[%s], predicted=[%s]' % (raw_src, raw_target, translation))
		actual.append([raw_target.split()])
		predicted.append(translation.split())
	# calculate BLEU score
	print('BLEU-1: %f' % corpus_bleu(actual, predicted, weights=(1.0, 0, 0, 0)))
	print('BLEU-2: %f' % corpus_bleu(actual, predicted, weights=(0.5, 0.5, 0, 0)))
	print('BLEU-3: %f' % corpus_bleu(actual, predicted, weights=(0.3, 0.3, 0.3, 0)))
	print('BLEU-4: %f' % corpus_bleu(actual, predicted, weights=(0.25, 0.25, 0.25, 0.25)))

In [ ]:
from keras.models import load_model 
import numpy as np
model = load_model('model.h5')
# test on some training sequences
print('train')
evaluate_model(model, eng_tokenizer, trainX, train)
# test on some test sequences
print('test')
evaluate_model(model, eng_tokenizer, testX, test)

train
src=[['톰', '은', '해외', '에서', '공부', '하기로', '결심', '했대']], target=[['tom', 'has', 'decided', 'to', 'study', 'abroad']], predicted=[i i to to]


AttributeError: ignored